In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Define the range of file names from 377 to 1896
file_range = range(377, 1897)

# Initialize lists to store data
target_id_g = []
mag_g = []
magerr_g = []

target_id_r = []
mag_r = []
magerr_r = []

target_id_i = []
mag_i = []
magerr_i = []

# Use tqdm to create a progress bar
for x in tqdm(file_range):
    file_path = f"{x:06d}.hdf5"  # Format the file name with leading zeros

    try:
        with h5py.File(file_path, 'r') as file:
            keys = list(file.keys())
            for key in keys:
                if 'g' in file[key] and 'mag' in file[key]['g'] and 'magerr' in file[key]['g']:
                    indices = (file[key]['g']['catflags'][:] & 32768) == 0
                    target_id_g.append(key)
                    mag_g.extend(file[key]['g']['mag'][indices])
                    magerr_g.extend(file[key]['g']['magerr'][indices])
                if 'r' in file[key] and 'mag' in file[key]['r'] and 'magerr' in file[key]['r']:
                    indices = (file[key]['r']['catflags'][:] & 32768) == 0
                    target_id_r.append(key)
                    mag_r.extend(file[key]['r']['mag'][indices])
                    magerr_r.extend(file[key]['r']['magerr'][indices])
                elif 'i' in file[key] and 'mag' in file[key]['i'] and 'magerr' in file[key]['i']:
                    indices = (file[key]['i']['catflags'][:] & 32768) == 0
                    target_id_i.append(key)
                    mag_i.extend(file[key]['i']['mag'][indices])
                    magerr_i.extend(file[key]['i']['magerr'][indices])
    except FileNotFoundError:
        print(f"File not found: {file_path}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

NameError: name 'tqdm' is not defined

In [3]:
def mag_flux(magnitude,uncertainties):
    f = 10 ** (-magnitude / 2.5)
    df = f * uncertainties * np.log(10) / (-2.5)
    return f,df

In [4]:
result1 = [mag_flux(magnitudes_g[x],uncertainties_g[x]) for x in range(len(magnitudes_g))]
result2 = [mag_flux(magnitudes_r[x],uncertainties_r[x]) for x in range(len(magnitudes_r))]
result3 = [mag_flux(magnitudes_i[x],uncertainties_i[x]) for x in range(len(magnitudes_i))]

f_g = [x[0] for x in result1]
df_g = [x[1] for x in result1]

f_r = [x[0] for x in result2]
df_r = [x[1] for x in result2]

f_i = [x[0] for x in result3]
df_i = [x[1] for x in result3]

In [5]:
def weighted_average(f, df):
    # Calculate the weights
    weights = 1.0 / (df ** 2)
    
    # Calculate the weighted average magnitude
    weighted_flux = np.sum(f * weights) / np.sum(weights)

    # Calculate the weighted uncertainties
    weighted_uncertainties = np.sqrt(1.0 / np.sum(weights))

    return weighted_flux, weighted_uncertainties

In [6]:
result1 = [weighted_average(f_g[x],df_g[x]) for x in range(len(f_g)) ]
result2 = [weighted_average(f_r[x],df_r[x]) for x in range(len(f_r)) ]
result3 = [weighted_average(f_i[x],df_i[x]) for x in range(len(f_i)) ]



#print(len(result))
weighted_ave_f_g = [x[0] for x in result1]
weighted_ave_df_g = [x[1] for x in result1]

weighted_ave_f_r = [x[0] for x in result2]
weighted_ave_df_r = [x[1] for x in result2]

weighted_ave_f_i = [x[0] for x in result3]
weighted_ave_df_i = [x[1] for x in result3]
#print(weighted_ave_mag_g)
f_g = np.array([weighted_ave_f_g])
df_g = np.array([weighted_ave_df_g])

f_r = np.array([weighted_ave_f_r])
df_r = np.array([weighted_ave_df_r])

f_i = np.array([weighted_ave_f_i])
df_i = np.array([weighted_ave_df_i])

mag_g = -2.5 * np.log10(f_g)
mag_r = -2.5 * np.log10(f_r)
mag_i = -2.5 * np.log10(f_i)
magerr_g = -2.5 / np.log(10) * df_g / f_g
magerr_r = -2.5 / np.log(10) * df_r / f_r
magerr_i = -2.5 / np.log(10) * df_i / f_i

In [7]:
data = pd.DataFrame({'Targetid':target_id,
                     'mag_g': weighted_ave_mag_g[0],
                    'magerr_g':weighted_ave_magerr_g[0],
                    'mag_r': weighted_ave_mag_r[0],
                    'magerr_r':weighted_ave_magerr_r[0],
                    'mag_i': weighted_ave_mag_i[0],
                    'magerr_i':weighted_ave_magerr_i[0],})
csv_file_path = "weighted_average.csv"
# Save the data to a CSV file
data.to_csv(csv_file_path, index=False)